In [30]:
import pandas as pd
from datetime import datetime

%matplotlib inline

In [31]:
cities = pd.read_csv('CreditCardTransactions/cities.csv')
customers = pd.read_csv('CreditCardTransactions/customers.csv')
merchants = pd.read_csv('CreditCardTransactions/merchants.csv')
transactions = pd.read_csv('CreditCardTransactions/test_transactions.csv')

# Merge do customers.csv e cities.csv usando 'city'
customers_cities = pd.merge(customers, cities, on='city', how='left')

# Merge do transactions.csv e customers_cities usando 'cc_num'
transactions_customers = pd.merge(transactions, customers_cities, on='cc_num', how='left')

# Merge final do transactions_customers e merchants.csv usando 'merchant'
final_data = pd.merge(transactions_customers, merchants, on='merchant', how='left')

# Guardar o resultado final
final_data.to_csv('CreditCardTransactions/merged_teste_dataset.csv', index=False)

data = pd.read_csv('CreditCardTransactions/merged_teste_dataset.csv')

new_data = pd.read_csv('new_data.csv')

X_train = pd.read_csv('CreditCardTransactions/training_dataset.csv')
y_train = pd.read_csv('CreditCardTransactions/training_classes.csv')

In [32]:
data['num_na'] = data.isna().sum(axis=1)
df_unico = data.loc[data.groupby('trans_num')['num_na'].idxmin()].reset_index(drop=True)
data = df_unico.drop(columns=['num_na'])
all_data = data.copy()

df = data.copy()
data = data.sort_values(by=['unix_time'])
data['transactions_count'] = (
    data.groupby(['cc_num', 'merchant'])['unix_time']
      .cumcount()
)
data['transactions_count'] = data['transactions_count'] + 1 

all_data.sort_values(by=['amt'], inplace=True)

data['dob'] = pd.to_datetime(data['dob'])
today = datetime.today()
data['dob'] = data['dob'].apply(lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)))


data = data.drop(columns= ['index', 'first', 'last', 'lat', 'long', 'merch_lat', 'merch_long', 'merchant_id', 'trans_date_trans_time', 'street', 'zip', 'trans_num', 'merchant', 'cc_num'])

numerical_cols = new_data.select_dtypes(include=['number']).columns
categorical_cols = new_data.select_dtypes(include=['object', 'category']).columns
mean_values = new_data[numerical_cols].mean()
mode_values = new_data[categorical_cols].mode().iloc[0]
data[numerical_cols] = data[numerical_cols].fillna(mean_values)
data[categorical_cols] = data[categorical_cols].fillna(mode_values)

data.head()

X_test = pd.get_dummies(data, drop_first=False, dtype=int)
'''
X_test['device_os_Windows'] = 0
colunas = X_test.columns.tolist()
colunas.insert(5, colunas.pop(colunas.index('device_os_Windows')))
X_test = X_test[colunas]
X_test = X_test.rename(columns={'device_os_macOs': 'device_os_macOS'})
'''

"\nX_test['device_os_Windows'] = 0\ncolunas = X_test.columns.tolist()\ncolunas.insert(5, colunas.pop(colunas.index('device_os_Windows')))\nX_test = X_test[colunas]\nX_test = X_test.rename(columns={'device_os_macOs': 'device_os_macOS'})\n"

In [33]:
X_test.head()

,amt,unix_time,dob,city_pop,transactions_count,device_os_Windows,device_os_X11,device_os_macOs,device_os_other,gender_F,...,state_AZ,state_CA,state_IL,state_NY,state_TX,category_Apparel,category_Electronics,category_Entertainment,category_Groceries,category_Travel
625,347.888981,1672531200,25,2328000.0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1149,27.465953,1672531200,46,1680992.0,1,1,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
1379,131.935282,1672531200,35,3979576.0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
1130,272.724032,1672531200,42,1680992.0,1,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
942,115.586445,1672531200,55,1680992.0,1,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [34]:
all_data.head(5)

,index,trans_date_trans_time,cc_num,device_os,merchant,amt,trans_num,unix_time,first,last,...,job,dob,lat,long,city_pop,state,category,merch_lat,merch_long,merchant_id
1096,30447,2023-08-19 00:00:00,5552705659737069,NaN,Merchant_92,5.000587,TR569729,1702425600,Alice,Clark,...,Lawyer,2002-03-07,34.0522,-118.2437,3979576.0,CA,Groceries,-63.539476,-101.993026,92
1811,30080,2023-09-01 00:00:00,4235959476852339,macOs,Merchant_100,5.396069,TR910867,1688256000,Mike,Smith,...,Doctor,1994-11-26,40.7128,-74.0060,8419600.0,NY,Travel,-71.477291,-30.293394,100
222,30201,2023-07-31 00:00:00,4095489114638262,NaN,Merchant_17,5.521485,TR195042,1702771200,Mike,Johnson,...,NaN,1955-05-11,40.7128,-74.0060,8419600.0,NY,Entertainment,86.430205,108.187643,17
1828,30118,2023-06-08 00:00:00,9799421716507981,NaN,Merchant_98,5.632781,TR921159,1699315200,Mike,Wilson,...,Teacher,1965-11-22,34.0522,-118.2437,3979576.0,CA,Groceries,29.759642,-16.648430,98
776,30160,2023-06-20 00:00:00,2923383360836086,NaN,Merchant_48,6.402435,TR432020,1679616000,Emily,Smith,...,Teacher,1970-03-23,34.0522,-118.2437,3979576.0,CA,Apparel,-34.824301,9.043277,48


In [35]:
X_test.sort_values(by=['amt'], inplace=True)
X_test.head(5)

,amt,unix_time,dob,city_pop,transactions_count,device_os_Windows,device_os_X11,device_os_macOs,device_os_other,gender_F,...,state_AZ,state_CA,state_IL,state_NY,state_TX,category_Apparel,category_Electronics,category_Entertainment,category_Groceries,category_Travel
1096,5.000587,1702425600,22,3979576.0,1,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
1811,5.396069,1688256000,30,8419600.0,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,1
222,5.521485,1702771200,69,8419600.0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1828,5.632781,1699315200,59,3979576.0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
776,6.402435,1679616000,54,3979576.0,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [36]:
from sklearn.tree import DecisionTreeClassifier
best_decision_tree = DecisionTreeClassifier(max_depth = 10, max_features = 10, random_state = 42)
best_decision_tree.fit(X_train, y_train)

# 6. Fazer previsões
y_pred = best_decision_tree.predict(X_test)

# 7. Avaliar o modelo
#accuracy = accuracy_score(y_test, y_pred)

resultados = best_decision_tree.predict_proba(X_test)[:, 1] 

submission = pd.DataFrame({
    'index': all_data['index'],
    'is_fraud': resultados  
})

submission.to_csv('tentativas/teste1.csv', index=False)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- category_Apparel
- city_Chicago
- device_os_macOs
- gender_F
- job_Artist
- ...
Feature names seen at fit time, yet now missing:
- device_os_macOS
